Основные проблемы, с которыми придется столкнуться:
* <b>сервер выдает только первые 200 записей</b>, соответствующих запросу, даже если их 200 000. Это значит, что данные придется получать кусками по 200 строк, эти 200 строк записывать, вводить новый запрос, заново нажимать кнопку, получать новые 200 строк и опять писать их в файл. Поэтому процесс затягивается.
* <b>время отработки запроса сервером</b>. Оно разнится от 0,2 секунд до 5 секунд (>5 секунд - timeout, про это в следующем пункте). Это ведет к тому, что у нас есть 2 варианта, как разворачивать программу:
а) или на каждой итерации ждать ровно 5 секунд в ожидании, пока программа 100% отработает запрос;
б) или ждать, предположим, 1 секунду, потом проверять, отработан ли запрос, если нет, то ждать ещё 1 секунду и опять проверять, и так до победного (выбираем этот вариант, т.к. мы сэкономим по 4 секунды на запросах, которые отработались моментально и сэкономим максимум из возможного времени, проверяя результат каждые 1-2 секунды, вместо всех 5).
* <b>timeout</b> - то, что вываливается, когда сервер не смог в отведенные 5000 мс отработать запрос. По моим наблюдениям, это происходит нередко тогда, когда мы парсим данные в 2 потока (например, 1 поток парсит данные, начиная с начала таблицы, 2 - начиная с конца), тогда, когда оба потока в один момент отправляют запрос в базу данных, база данных и так медленно их отрабатывает, а их тут ещё и два, всё вываливается в timeout. При однопоточном режиме такие ситуации практически исключены. Остановился на варианте с одним потоком, так как два потока не дают практически никакого прироста производительности, но увеличивают риски нарваться на timeout, а это ведёт к увеличению среднего времени работы по каждой итерации. Тем не менее, нам необходимо предусмотреть эту ситуацию.
<br/><br/><br/>
Как выглядит процесс:

<img src="https://lh3.googleusercontent.com/u9Os8sHHFc91WuYKWFDGgYgXyjDEG5Ei53g84OQzFNT0B075jA_M2hgiGu_Gt6YD38QxaLXQcLiFZ5zR1MuriQKfNhzI6VsJ0u8O540e-PcxxCHW9V0xBTS6ddMoJpVpes_xmhLIcaZutwZH-ipUCb10Gf72CrhznvGWVlwbfxxV40paqNvwZ3Ka__IVXig-G6lxb4i6fKymsntqf3JpD-tj_nV4121ABnDbwAJUJPnGfS3-I_T99jvRR23dpQ9MgagS-QPJ0ACmy4zvYfhWc7VHIUZ7SnDrTmloyRnm6YRsPLbn_PW6_v07onGBpxUd7aKhTq27wz9KyH4ZkNL5lET5wYjESoZG_s-zhKFGcp1yU-O26jRFzn5hJokXTUrx1Ys9JzI-zRPFJ5__h1ks2yGOnzgs1W2VKtl4luyeIR789Dt6K0liRpWOPwRLZbn5b6ME2rBE9GLogOVF06QZlbxnFWF5XOAYHS9UhDEfQ4Fkwvh004WnydVjou9P1Nc1EFPAuoQ-nkhFBWS7C91VTFnFueb1vemP8MZ7aLuZYYzEPJtUyqacYtemWV2QtnsU3u7Qv_18ER729UrF9jDuz9ZQnfjvbT-lFpFp9y4EzOjkGHoRuICeORPah53EF3OC5Q1_XSHXSh3haewMpuGGaVPljriHYTzWx3a14JJijEYAVEGN0AAOqA=w1189-h820-no" alt="альтернативный текст">

Установим (если не установлена) библиотеку:

In [6]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import csv
from tqdm import tqdm_notebook
import numpy as np

caps = webdriver.DesiredCapabilities().FIREFOX
caps["marionette"] = True
browser1 = webdriver.Firefox(executable_path=r'E:\Chrome\geckodriver.exe', capabilities=caps)
browser1.get("https://praktikum.yandex.ru/trainer/data-analyst/lesson/9bd17813-3d5c-47da-ab3c-097baced1979/task/03fbb4b3-ba91-4660-a05e-7b181f597407/")

# для ввода логина-пароля и очистки поля ввода запроса
time.sleep(15)

# ищем поле ввода sql-запроса и кнопку "play"
inputElement1 = browser1.find_elements_by_tag_name('textarea')[0]
button1 = browser1.find_elements_by_xpath("//button[@class='button button_has-hover-color button_size_m button_type_icon button_theme_light button_view_clear']")[0]

# пишем первый запрос и спим на всякий случай 
# (БД иногда плохо реагирует на почти одновременный ввод запроса и нажатие кнопки)
inputElement1.send_keys("select *\nfrom flights")
time.sleep(1)

# проверяем, есть ли вывод (когда вывод не подгрузился - кнопка ещё не активна)
# кнопку на "активность" и проверяем
try:
    button1.click()
except:
    try:
        time.sleep(1)
        button1.click()
    except:
        try:
            time.sleep(1)
            button1.click()
        except:
            pass
    
# ждем вывода результата    
time.sleep(5)

# парсим данные и ищем таблицу с результатом
parser1 = BeautifulSoup(browser1.page_source,"lxml")
table1 = parser1.find("table")

# пишем в файл
with open('flights.csv','w+', newline='') as csvfile1:
    writer1 = csv.writer(csvfile1, delimiter=',')
    
    for tr in table1.findAll("tr")[1:]:
        list_of_cells1 = list()
        for td in tr.findAll("td"):
            list_of_cells1.append(td.text)
        writer1.writerow(list_of_cells1)

# количество циклов в зависимости от общего количества строк в БД    
len_of_db = 65664 // 200

# запускаем цикл для ввода sql-запросов уже с offset и парсинга результатов
for i in tqdm_notebook(range(len_of_db)):
    # сохраняем таблицу, спарсенную в предыдущей итерации (для последующей сверки)
    prev_table1 = table1
    # удаляем последние символы после offset в определенном количестве
    if i >= 1:
        chars_to_delete = len(str(i*200))
        for delete_count in range(chars_to_delete):
            inputElement1.send_keys(Keys.BACKSPACE) 
    i += 1
    # дополняем sql-запрос
    if i > 1:
        inputElement1.send_keys("{}".format(i*200))
    else:
        inputElement1.send_keys("\noffset {}".format(i*200))
        
    time.sleep(0.5)
    # нажимаем кнопку для получения результата 
    # (иногда это получается не с первого раза, поэтому try-except)
    try:
        button1.click()
    except:
        try:
            time.sleep(0.5)
            button1.click()
        except:
            try:
                time.sleep(0.5)
                button1.click()
            except:
                pass
    
    # спим в ожидании вывода результата
    time.sleep(2)
    
    # проверка, точно ли нам выдались результаты 
    # (если "нет", кнопка будет неактивна, если "да", то уже сохраненный в кэш резуьтат будет выведен заново моментально)
    try:
        button1.click()
    except:
        try:
            time.sleep(1)
            button1.click()
            time.sleep(2)
        except:
            try:
                time.sleep(1)
                button1.click()
                time.sleep(2)
            except:
                pass
            
    # получаем таблицу с данными
    parser1 = BeautifulSoup(browser1.page_source,"lxml")
    table1 = parser1.find("table")
    # счетчики, сколько раз мы спарсили ту же таблицу (долго грузилась новая) и сколько раз был timeout
    prev_counter = 0
    none_counter = 0
    
    # цикл "до победного" (до получения нужной таблицы)
    # на случай, если оказалось, что мы спарсили ту же таблицу, что и в итерации ранее,
    # или когда у нас timeout, и таблица - пустой объект:
    while table1 == prev_table1 or table1 is None:
        if table1 == prev_table1:
            reason = 'повтор'
            prev_counter += 1
        elif table1 is None:
            reason = 'timeout'
            none_counter += 1
            
        print('Я тут, причина - {} в {} раз'.format(reason, [none_counter, prev_counter][reason == 'повтор']))
        
        time.sleep(2)
        try:
            button1.click()
            time.sleep(1)
        except:
            try:
                time.sleep(1)
                button1.click()
                time.sleep(1)
            except:
                pass
        parser1 = BeautifulSoup(browser1.page_source,"lxml")
        table1 = parser1.find("table")
    
    # записываем очередную порцию данных в файл
    with open('flights.csv','a', newline='') as csvfile1:
        writer1 = csv.writer(csvfile1, delimiter=',')

        for tr in table1.findAll("tr")[1:]:
            list_of_cells1 = list()
            for td in tr.findAll("td"):
                list_of_cells1.append(td.text)
            writer1.writerow(list_of_cells1)